# Exploring `scope`

M. Gully-Santiago  
June 24, 2019
Kepler/K2 GO Office

In [ ]:
import scope

In [ ]:
import lightkurve as lk

## Stellar lightcurves

In [ ]:
target = scope.generate_target()

In [ ]:
target

In [ ]:
# %load /Users/obsidian/Desktop/defaults.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
plt.plot(target.xpos, target.ypos, '.')

Neat

In [ ]:
tpf = target.targetpixelfile

In [ ]:
tpf.shape

In [ ]:
import copy

# Simulate test data:

>synthetic-k2-flat.targ.fits shall use a flat light curve;

> synthetic-k2-sinusoid.targ.fits shall use a low-frequency sinusoid (e.g. 5-day period);

>synthetic-k2-planet.targ.fits shall use a short-period Neptune-sized planet (e.g. Kp = 14; planet period = 5 days; transit depth = 2000ppm).

In [ ]:
target_flat = copy.copy(target)

In [ ]:
target_flat

In [ ]:
target_sine = copy.copy(target)

In [ ]:
target_sine

In [ ]:
rot_period = 5.0 # days
lc_amp = 0.001 # percent amplitude
injected_sine_lc = 1.0+ lc_amp * np.sin(2.0*np.pi*target.t/rot_period)

In [ ]:
plt.plot(target.t, injected_sine_lc)

In [ ]:
target_sine.add_variability(custom_variability=injected_sine_lc)

In [ ]:
target_sine.plot()

Yay, looks good

In [ ]:
sine_tpf = target_sine.to_lightkurve_tpf(target_id="simulated_sine_P5days_Amp0p001")

In [ ]:
sine_tpf.get_keyword('2CRV5P', hdu=1, default=0)

### Synthetic TPFs lack requisite metadata found in genuine TPFs

We want to generate synthetic TPFs that satisfy three sometimes-competing demands:
- resemble genuine TPFs in every important way
- possess the least amount of placeholder metadata so as not to confuse synthetic with real data.
- possess the pristine injected signal vector and affiliated metadata


Let's **manually** add these parcels of metadata:
 - row
 - column

In [ ]:
genuine_tpf = lk.search_targetpixelfile(205998445, mission='K2').download()

In [ ]:
genuine_tpf.plot(aperture_mask=genuine_tpf.pipeline_mask)

Compare the genuine TPF header to the synthetic TPF header.

In [ ]:
col = genuine_tpf.get_keyword('1CRV5P', hdu=1, default=0)
row = genuine_tpf.get_keyword('2CRV5P', hdu=1, default=0)

sine_tpf.hdu[1].header.set('1CRV5P', value=col)
sine_tpf.hdu[1].header.set('2CRV5P', value=row)

Save the TPF for round-trip checking.

In [ ]:
sine_tpf.to_fits(output_fn='temp.fits', overwrite=True)

In [ ]:
tpf_roundtrip = lk.KeplerTargetPixelFile('temp.fits')

tpf_roundtrip.plot()

Yay, it works.

In [ ]:
lc = tpf_roundtrip.to_lightcurve()

In [ ]:
lc.plot(normalize=False)
plt.plot(target_sine.)

In [ ]:
target_sine.